# Neural Network Prediction of Sahelian Summer Rainfall
***

#### Resources:
* [Mardata Course](https://github.com/mardatade/Course-Python-for-Machine-Learning/blob/master/3.%20Neural%20Network.ipynb)
* [Keras for Data Scientists](https://keras.io/getting_started/intro_to_keras_for_engineers/#data-loading-amp-preprocessing)

In [ ]:
import numpy as np
import pandas as pd 
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as st

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn import metrics


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import tensorflow_addons as tfa

from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard


from dask import delayed

<br>
<br>

## 1. Data Loading & Preprocessing
***

<br>

### a) Loading & Normalization

**predictor:** contains the data used for the inputs  
**label:** from Sahelrainfall data serves as validation data

In [ ]:
predictor = xr.open_dataset('data/da_pred_all.nc').to_dataframe()

predictor_unit = pd.DataFrame(
    data = StandardScaler().fit_transform(predictor), 
    columns = predictor.columns,
    index =  predictor.index
)


# load validatoin data (Summer Rainfall over Sahel and scale to [cm/month]) 
labels = np.mean(np.loadtxt("data/da_o_sahelprecip19012017.txt", skiprows=8,)[:,7:10] * 0.01,  axis=1)

predictor_unit.head()

<br>

### b) PCA

In [ ]:
# Scikit PCA transformation
pca = PCA()
principalComponents = pca.fit_transform(predictor_unit)


# Create Create Pandas DF from PCs
col = []
for i in range(1, 21):
    col.append(f'PC{i}')

predictor_pc = pd.DataFrame(
    data = principalComponents,
    columns = col,
    index =  predictor.index
)

# Test for unit-variance and zero mean:
# np.std(pred_pc)
# np.mean(pred_pc)
# pred_pc.head()

predictor_pc.head()

<br>
<br>

## 2. MODEL SETUP AND TUNING
***

<br>

### Set Log Parent Directory
***

##### Set Parent Directory

In [ ]:
parent_dir = 'logs/tf_no dask/'

##### Clear Directory

In [ ]:
# rm -rf logs/tf_nodask/*

<br>

### Hyperparameter Selection
***

In [ ]:
###################################
#####EXAMPLE SETUP FOR TESTING#####
###################################


#GRID SERACH HYPERPARAMETER#
#---------------------------
HP_INPUT_VAR_NINE = hp.HParam('input_var_nine', hp.Discrete(['PC9', 'PC14']),display_name='9th Input Variable')
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['AdamW', 'SGDW']),display_name='Optimizer')
HP_LEARN_RATE = hp.HParam('learn_rate', hp.Discrete([0.1]),display_name='Learning Rate')
HP_WEIGHT_DECAY = hp.HParam('weight_decay', hp.Discrete([1e-1]),display_name='Weight Decay')
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1]),display_name='Batch Size')
HP_EPOCHS = hp.HParam('n_epochs', hp.Discrete([3]),display_name='Epochs')


#CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
#----------------------------------------------------
cv_param={
    'N_FOLDS': 2,         # number of folds -> small for Test Runs
    'TEST_FRAC': .1    # factrion that is held out for test
}


#BAGGING PARAMETER (NO PART OF GRID SEARCH)#
#-------------------------------------------
n_baggs = 2  # number of baggs -> small for test runs





####################
#####FULL SETUP#####
####################


# #GRID SERACH HYPERPARAMETER#
# #---------------------------
# HP_INPUT_VAR_NINE = hp.HParam('input_var_nine', hp.Discrete(['PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16']),display_name='9th Input Variable')
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['AdamW', 'SGDW']),display_name='Optimizer')
# HP_LEARN_RATE = hp.HParam('learn_rate', hp.Discrete([0.01, 0.1, 0.2]),display_name='Learning Rate')
# HP_WEIGHT_DECAY = hp.HParam('weight_decay', hp.Discrete([0.001, 0.01, 0.1]),display_name='Weight Decay')
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1, 3, 10, 30]),display_name='Batch Size')
# HP_EPOCHS = hp.HParam('n_epochs', hp.Discrete([30, 80, 120]),display_name='Epochs')


# #CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
# #----------------------------------------------------
# cv_param={
#     'N_FOLDS': 105,      # number of folds -> sample size as in Badr
#     'TEST_FRAC': .1    # factrion that is held out for test
# }


# #BAGGING PARAMETER (NO PART OF GRID SEARCH)#
# #-------------------------------------------
# n_baggs = 10 # number of baggs -> 10 as in Badr

<br>

### Metric Selection
---

In [ ]:
METRIC_TRAIN_MSE_MU= 'train_mse_mu'
METRIC_TRAIN_MSE_SIG= 'train_,mse_sig'
METRIC_TRAIN_CORR_MU= 'train_corr_mu'
METRIC_TRAIN_CORR_SIG= 'train_corr_sig'

METRIC_TEST_MSE_MU= 'test_mse_mu'
METRIC_TEST_MSE_SIG= 'test_mse_sig'
METRIC_TEST_CORR_MU= 'test_corr_mu'
METRIC_TEST_CORR_SIG= 'test_corr_sig'

<br>

### Log Experiment Confiuration to TensorBoard
---

In [ ]:
with tf.summary.create_file_writer(parent_dir).as_default():
    hp.hparams_config(
        hparams=[HP_INPUT_VAR_NINE, HP_OPTIMIZER, HP_LEARN_RATE, HP_WEIGHT_DECAY, HP_BATCH_SIZE, HP_EPOCHS],
        metrics=[
            hp.Metric(METRIC_TRAIN_MSE_MU, display_name='Training Sample MSE µ'),
            hp.Metric(METRIC_TRAIN_MSE_SIG, display_name='Training Sample  MSE σ'),
            hp.Metric(METRIC_TRAIN_CORR_MU, display_name='Training Sample Correlation µ'),
            hp.Metric(METRIC_TRAIN_CORR_SIG, display_name='Training Sample  Correlation σ'),
            hp.Metric(METRIC_TEST_MSE_MU, display_name='Test Sample MSE µ'),
            hp.Metric(METRIC_TEST_MSE_SIG, display_name='Test Sample  MSE σ'),
            hp.Metric(METRIC_TEST_CORR_MU, display_name='Test Sample Correlation µ'),
            hp.Metric(METRIC_TEST_CORR_SIG, display_name='Test Sample  Correlation σ')
        ],
    )

<br>

### Build Model Function
---

In [ ]:
def BuildModel(hparams):      
    
    
    model = keras.Sequential([
            layers.Dense(3, activation="sigmoid", name="layer1", input_shape=(9,)),
            layers.Dense(1, activation='linear', name='output')
        ])
    
    model.compile(
        loss='mean_squared_error',
        optimizer=getattr(tfa.optimizers, hparams[HP_OPTIMIZER])(
            learning_rate=hparams[HP_LEARN_RATE],
            weight_decay=hparams[HP_WEIGHT_DECAY]
        )
    )
    return model

<br>

### Bagging Function
---

In [ ]:
def Bagging(hparams, features, model, train_index, test_index):
    
    
    
    # set emty output matrices
    y_train_bagging = np.zeros((train_index.size, n_baggs))
    y_test_bagging = np.zeros((test_index.size, n_baggs))    
    
    
    #Train the model 'n_baggs' times and store model predictions into matrice
    for n in range(n_baggs):
        
#         print ('baggin run', n)
#         print ('PREDICTION ON TEST DATA:', y_test_bagging)
        
        # Bootstrap sampling from training Data with Size(Training Data)
        train_index_bootstrap = np.random.choice(train_index, train_index.size)

        #Train the model 
        model.fit(
            features[train_index_bootstrap],
            labels[train_index_bootstrap],
            batch_size=hparams[HP_BATCH_SIZE],
            epochs=hparams[HP_EPOCHS],
            verbose=1
        )
        
        #Run the model for insample data and store in one matrix:
        y_train_bagging[:, n] = np.squeeze(model.predict(features[train_index]))
        
        # ... and for out of sample data        
        y_test_bagging[:, n] = np.squeeze(model.predict(features[test_index]))

    #return mean of the outputs over baggins (1st dimension)
    return y_train_bagging.mean(1), y_test_bagging.mean(1)

<br>

### Cross Validation Training & Error Calculation Funktion
---

In [ ]:
def TrainModel(hparams, cv_param, predictor_pc, labels):
        
    
    train_mse = np.empty(cv_param['N_FOLDS'])
    train_corr = np.empty(cv_param['N_FOLDS'])
    
    test_mse = np.empty(cv_param['N_FOLDS'])
    test_corr = np.empty(cv_param['N_FOLDS'])
    
    #choose Inputs
    features = predictor_pc.loc[:,['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', hparams[HP_INPUT_VAR_NINE]]].to_numpy()
    
    
    
    
    #Cross Validation#
    ##################
    
    cv_fold = 0
    
    for train_index, test_index in ShuffleSplit(n_splits=cv_param['N_FOLDS'], test_size=cv_param['TEST_FRAC']).split(features):
        
#         print(cv_fold)
#         print("TRAIN:", train_index, "TEST:", test_index)
        
    
        # Build the model according to definition:
        model = BuildModel(hparams)
        
        # Train and predict using Bagging    
        y_train, y_test = Bagging(hparams, features, model, train_index, test_index)
        
        
        #Compute error metrics for in sample data
        train_err=  y_train - labels[train_index]
        train_mse[cv_fold] = np.mean(train_err**2)
        train_corr[cv_fold] = st.pearsonr(y_train, labels[train_index])[0]
        
        # ... and for out of sample data
        test_err=  y_test - labels[test_index]
        test_mse[cv_fold] = np.mean(test_err**2)
        test_corr[cv_fold] = st.pearsonr(y_test, labels[test_index])[0]
        
        
#         print ( "BAGGING OUT Test", y_test)
        cv_fold += 1
    
    #######################################################################################################
    
    
    #Error Moments#
    ###############
    
    eval_metrics = {
        'train_mse_mu': np.mean(train_mse),
        'train_mse_sig': np.std(train_mse),
        'train_corr_mu': np.mean(train_corr),
        'train_corr_sig': np.std(train_corr),            
        'test_mse_mu': np.mean(test_mse),
        'test_mse_sig': np.std(test_mse),
        'test_corr_mu': np.mean(test_corr),
        'test_corr_sig': np.std(test_corr),
    }
    
#     print(eval_metrics)
    
    return eval_metrics

<br>

### Model Run and Log Function
---

In [ ]:
def RunModel(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        
        eval_metrics = TrainModel(hparams, cv_param, predictor_pc, labels)
        eval_metrics.a()
        tf.summary.scalar(METRIC_TRAIN_MSE_MU,   eval_metrics['train_mse_mu'],   step=1)
        tf.summary.scalar(METRIC_TRAIN_MSE_SIG,  eval_metrics['train_mse_sig'],  step=1)
        tf.summary.scalar(METRIC_TRAIN_CORR_MU,  eval_metrics['train_corr_mu'],  step=1)
        tf.summary.scalar(METRIC_TRAIN_CORR_SIG, eval_metrics['train_corr_sig'], step=1)
        tf.summary.scalar(METRIC_TEST_MSE_MU,    eval_metrics['test_mse_mu'],    step=1)
        tf.summary.scalar(METRIC_TEST_MSE_SIG,   eval_metrics['test_mse_sig'],   step=1)
        tf.summary.scalar(METRIC_TEST_CORR_MU,   eval_metrics['test_corr_mu'],   step=1)
        tf.summary.scalar(METRIC_TEST_CORR_SIG,  eval_metrics['test_corr_sig'],  step=1)

<br>

### Grid Search
---

In [ ]:
session_num = 0        

for input_var_nine in HP_INPUT_VAR_NINE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for learn_rate in HP_LEARN_RATE.domain.values:
            for weight_decay in HP_WEIGHT_DECAY.domain.values:
                for batch_size in HP_BATCH_SIZE.domain.values:
                    for n_epochs in HP_EPOCHS.domain.values:


                        hparams = {
                            HP_INPUT_VAR_NINE: input_var_nine,
                            HP_OPTIMIZER: optimizer,
                            HP_LEARN_RATE: learn_rate,
                            HP_WEIGHT_DECAY: weight_decay,
                            HP_BATCH_SIZE: batch_size,
                            HP_EPOCHS: n_epochs,                
                        }

                        run_name = f"run-{session_num:04d}"
#                         print(f'--- Starting trial: {run_name}')
#                         print({h.name: hparams[h] for h in hparams})
                        RunModel(parent_dir + run_name, hparams)
                        session_num += 1    

<br>
<br>

***
***
<br>
<br>